In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class SteepestDescentNumericalLr:
    def __init__(self): ...
    
    def optimize(self, 
                 func,  
                 x0, 
                 lr=0.01,
                 h=1e-5,
                 max_iterations=1000,
                 stop_value=None,
                 lr_schedule_type='fixed'):
        
        x = np.array(x0, dtype=float)
        trayectory = []
        initial_lr = lr
        for i in range(max_iterations):
            f_val = func(x)
            trayectory.append(f_val)

            if stop_value and f_val <= stop_value:
                    break

            grad = self.numerical_gradient(func, x, h=h)

            lr = self.get_learning_rate(
                schedule_type=lr_schedule_type,
                initial_lr=initial_lr,
                iteration=i,
                max_iterations=max_iterations,
                func=func,
                x=x,
                grad=grad
            )

            x = x - lr * grad
            
        return x, trayectory
    
    def numerical_gradient(self, f, x, h):
        n = len(x)
        grad = np.zeros(n)
        
        for i in range(n):
            x_plus = x.copy()
            x_minus = x.copy()
            
            x_plus[i] += h
            x_minus[i] -= h

            grad[i] = (f(x_plus) - f(x_minus)) / (2*h)

        return grad
    
    def get_learning_rate(self, schedule_type, initial_lr, iteration, max_iterations, func, x, grad):
        if schedule_type == 'fixed':
            return self.fixed_lr(initial_lr)
        elif schedule_type == 'linear_decreasing':
            return self.linear_decreasing_lr(initial_lr, iteration, max_iterations)
        elif schedule_type == 'adaptive':
            return self.adaptive_lr(initial_lr, iteration)
        elif schedule_type == 'intelligent':
            return self.intelligent_lr(func, x, grad, initial_lr)
        else:
            return initial_lr
    
    def fixed_lr(self, lr):
        return lr
    
    def linear_decreasing_lr(self, initial_lr, iteration, max_iterations):
        return initial_lr * (1 - iteration / max_iterations)
    
    def adaptive_lr(self, initial_lr, iteration):
        return initial_lr / (1 + 0.01 * iteration)

    def intelligent_lr(self, func, x, grad, initial_lr):
        alpha = initial_lr
        c1 = 1e-4
        rho = 0.5
        
        f_x = func(x)
        directional_derivative = -np.dot(grad, grad)

        while True:
            x_new = x - alpha * grad
            f_new = func(x_new)
            
            if f_new <= f_x + c1 * alpha * directional_derivative:
                break
            alpha *= rho
            
        return alpha
    
    def plot(self, trayectory, title="Descenso Más Pronunciado"):
        plt.figure(figsize=(8, 6))
        plt.plot(trayectory)
        plt.title(title)
        plt.xlabel('Iteraciones')
        plt.ylabel('Valor de la función')
        plt.grid(True)
        plt.show()

In [ ]:
def translated_sphere(x):
    c = np.ones(len(x))
    return np.sum((x - c)**2)

def rosenbrock(x):
    result = 0
    for i in range(len(x) - 1):
        result += 100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return result

def perm(x):
    B = 1
    result = 0
    for k in range(1, len(x) + 1):
        inner_sum = 0
        for i in range(1, len(x) + 1):
            inner_sum += (i + B) * (x[i-1]**k - (1/i)**k)
        result += inner_sum**2
    return result  

In [ ]:
n = 5
x0 = [0.5] * n
sd_algorithm = SteepestDescentNumericalLr()

In [ ]:
print("======= OPTIMIZACIÓN DE LA FUNCIÓN ESFERA TRASLADADA =======")

lr_strategies = {
    'fixed': 0.1, 
    'linear_decay': 0.1, 
    'adaptive': 0.1, 
    'armijo': 0.1
}
lr_names = ["TAMAÑO DE PASO FIJO", 
            "TAMAÑO DE PASO DECRECIENTE LINEALMENTE",
            "TAMAÑO DE PASO ADAPTATIVO", 
            "TAMAÑO DE PASO INTELIGENTE (BACKTRACKING)"]

trayectories = []

for schedule, lr, name in zip(lr_strategies.keys(), lr_strategies.values(), lr_names):
    x_sphere, trayectory_sphere = sd_algorithm.optimize(
        func=translated_sphere,
        x0=x0,
        lr=lr,
        h=1e-5,
        max_iterations=1000,
        stop_value=1e-2,
        lr_schedule_type=schedule
    )
    trayectories.append(trayectory_sphere)
    print("="*60)
    print(name)
    print("="*60)
    print("Valor óptimo encontrado:", x_sphere)
    print("Valor de la función en el óptimo:", translated_sphere(x_sphere))
    print("Número de iteraciones:", len(trayectory_sphere))

print("="*60)
print("Gráfica Descenso de la función:")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

for ax, history, name in zip(axs.flatten(), trayectories, lr_names):
    ax.plot(history, linewidth=2)
    ax.set_title(name)
    ax.set_xlabel('Iteraciones')
    ax.set_ylabel('Valor de la función')
    ax.grid(True)

plt.suptitle('Función De Esfera Trasladada - Comparación de Estrategias', fontsize=16)
plt.show()

In [ ]:
print("======= OPTIMIZACIÓN DE LA FUNCIÓN DE ROSENBROCK =======")

lr_strategies = {
    'fixed': 0.001, 
    'linear_decay': 0.001, 
    'adaptive': 0.0025, 
    'armijo': 0.001
}
lr_names = ["TAMAÑO DE PASO FIJO", 
            "TAMAÑO DE PASO DECRECIENTE LINEALMENTE",
            "TAMAÑO DE PASO ADAPTATIVO", 
            "TAMAÑO DE PASO INTELIGENTE (BACKTRACKING)"]

trayectories = []

for schedule, lr, name in zip(lr_strategies.keys(), lr_strategies.values(), lr_names):
    x_rosenbrock, trayectory_rosenbrock = sd_algorithm.optimize(
        func=rosenbrock,
        x0=x0,
        lr=lr,
        h=1e-5,
        max_iterations=1000,
        stop_value=1e-2,
        lr_schedule_type=schedule
    )
    trayectories.append(trayectory_rosenbrock)
    print("="*60)
    print(name)
    print("="*60)
    print("Valor óptimo encontrado:", x_rosenbrock)
    print("Valor de la función en el óptimo:", rosenbrock(x_rosenbrock))
    print("Número de iteraciones:", len(trayectory_rosenbrock))

print("="*60)
print("Gráfica Descenso de la función:")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

for ax, history, name in zip(axs.flatten(), trayectories, lr_names):
    ax.plot(history, linewidth=2)
    ax.set_title(name)
    ax.set_xlabel('Iteraciones')
    ax.set_ylabel('Valor de la función')
    ax.grid(True)

plt.suptitle('Función De Rosenbrock - Comparación de Estrategias', fontsize=16)
plt.show()

In [ ]:
print("======= OPTIMIZACIÓN DE LA FUNCIÓN DE PERM =======")

lr_strategies = {
    'fixed': 0.001, 
    'linear_decay': 0.001, 
    'adaptive': 0.001, 
    'armijo': 0.001
}
lr_names = ["TAMAÑO DE PASO FIJO", 
            "TAMAÑO DE PASO DECRECIENTE LINEALMENTE",
            "TAMAÑO DE PASO ADAPTATIVO", 
            "TAMAÑO DE PASO INTELIGENTE (BACKTRACKING)"]

trayectories = []

for schedule, lr, name in zip(lr_strategies.keys(), lr_strategies.values(), lr_names):
    x_perm, trayectory_perm = sd_algorithm.optimize(
        func=perm,
        x0=x0,
        lr=lr,
        h=1e-5,
        max_iterations=1000,
        stop_value=1e-2,
        lr_schedule_type=schedule
    )
    trayectories.append(trayectory_perm)
    print("="*60)
    print(name)
    print("="*60)
    print("Valor óptimo encontrado:", x_perm)
    print("Valor de la función en el óptimo:", perm(x_perm))
    print("Número de iteraciones:", len(trayectory_perm))

print("="*60)
print("Gráfica Descenso de la función:")
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

for ax, history, name in zip(axs.flatten(), trayectories, lr_names):
    ax.plot(history, linewidth=2)
    ax.set_title(name)
    ax.set_xlabel('Iteraciones')
    ax.set_ylabel('Valor de la función')
    ax.grid(True)

plt.suptitle('Función De Perm - Comparación de Estrategias', fontsize=16)
plt.show()